In [472]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score  
filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer 
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
pd.set_option("display.float_format", lambda x:"%.2f" % 4)
pd.set_option("display.max_rows", None)

In [453]:
df = pd.read_csv("../input/nlp-getting-started/train.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")

In [455]:
df.drop(["keyword","location"],axis=1,inplace=True)
test.drop(["keyword","location"],axis=1,inplace=True)

In [456]:
df["text"] = df["text"].str.lower()
df["text"] = df["text"].str.replace("[^\w\s]" , "")
df["text"]  = df["text"].str.replace("\d", "")
import nltk
sw = stopwords.words("english")
df["text"] = df["text"].apply(lambda x:" ".join(x for x in str(x).split() if x not in sw))
temp_df = pd.Series(" ".join(df["text"]).split()).value_counts()
drops = temp_df[temp_df < 20 ]
df["text"] = df["text"].apply(lambda x: " ".join(x for x in x.split() if x not in drops))
import nltk
nltk.download('omw-1.4')
df["text"] = df["text"].apply(lambda x: " ".join(Word(word).lemmatize() for word  in x.split()))
tf = df["text"].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis=0).reset_index()
tf.columns = ["words", "tf"]
nltk.download("vader_lexicon")
sia=SentimentIntensityAnalyzer()
df["polarity_scores"] = df["text"].apply(lambda x: sia.polarity_scores(x)["compound"])
df["sentiment_label"] = df["text"].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"]>0 else "neg")
df.sentiment_label = LabelEncoder().fit_transform(df["sentiment_label"])
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(0,1))
X_tf_idf_ngram_vectorizer = tf_idf_ngram_vectorizer.fit_transform(df["text"])
#vectorizer = CountVectorizer()
#X_Count = vectorizer.fit_transform(df["text"])

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [457]:
def protext(dataframe, col ):
    ### NORMALIZING CASE FOLDING
    dataframe[col] = dataframe[col].str.lower()
    #### PUNCTUATIONS
    dataframe[col] = dataframe[col].str.replace("[^\w\s]", "")
    dataframe[col] = dataframe[col].str.replace("\d", "")
    ### STOP WORDSLERİ KAYDEDİYORUM
    sw = stopwords.words("english")
    dataframe[col] = dataframe[col].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))
    ### RARE WORDS
    temp_df= pd.Series(" ".join(dataframe[col]).split()).value_counts()
    drops = temp_df[temp_df <=1]
    #### TOKENIZATION
    dataframe[col].apply(lambda x: TextBlob(x).words).head()
    ####LEMMATIZATION
    dataframe[col] = dataframe[col].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    test_tf = dataframe[col].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis=0).reset_index()
    test_tf.columns = ["words", "tf"]
    return dataframe.head()

In [458]:
y = df["target"]
X=X_tf_idf_ngram_vectorizer

In [459]:
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=17)

In [460]:
#rf_model = RandomForestClassifier(random_state=17)
#rf_params={"max_depth":[5,8,None],
           # "max_features" :[7,None],
           #"min_samples_split":[2,5,8],
           # "n_estimators":[100,200,300,400,500,600]}
#rf_best_grid = GridSearchCV(rf_model,rf_params, cv=5, n_jobs=-1, verbose=True).fit(X_train,y_train)

In [510]:
rf_model = RandomForestClassifier(n_estimators=20000,min_samples_split=5,max_features =19, random_state=17)
rf_final = rf_model.fit(X_train,y_train)

In [ ]:
cross_val_score(rf_final,X_train,y_train,cv=5, n_jobs=-1,verbose=True).mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [504]:
from sklearn.metrics import f1_score
y_pred= rf_final.predict(X_test)
score = f1_score(y_test, y_pred)  
print(score)  

0.7271573604060914


In [505]:
new = TfidfVectorizer().fit(df["text"]).transform(test["text"])
new_pred= rf_final.predict(new)

In [506]:
my_submission = pd.DataFrame({'id': test["id"], 'target': new_pred})
# you could use any filename. We choose submission here
my_submission.to_csv('sub9.csv', index=False)

In [466]:
#sub = pd.read_csv("./sub5.csv")

In [467]:
#sub.to_csv('submission__.csv', index=False)